# Analyzing Assessed Value of the Downtown Mall

## Goals: 

    *Provide descriptive statistics of assessed value over time for the City of Charlottesville's downtown mall

    *Map and chart assessed values over time

## Step 1: Aquire Data

### Assessment Values
    Charlottesville's Open Data Portal : http://opendata.charlottesville.org/

    Real Estate (All Assessments) Dataset : http://opendata.charlottesville.org/datasets/real-estate-all-assessments

        * On the Real Estate dataset page, in the upper right corner of the window under the map click the APIs             drop down
        * Copy the GeoJSON link
        * Use the GeoJSON link to pull data directly from the Open Data portal using the code below

    Parcel Area Data : http://opendata.charlottesville.org/datasets/parcel-boundary-area

### List of properties to use in analysis

Charlottesville GIS Viewer: https://gisweb.charlottesville.org/GisViewer/

    * Under Map option, turn on 'Parcels & Buildings' > 'Parcels'. Turn everything else off.

    * Zoom to area of interest on map

    * Under 'Tools' select 'Identify'

    * In 'Identify' toolbar select 'Custom Shape' and under 'Layer' select 'Parcels'

    * Using mouse on map, click boundary around area of interest

    * A list will appear in the left panel of the web page

    * In the panel click 'Tools' > 'Export All to Excel'

    * A window named 'Export Results' will open when your download is ready.

    * Click 'View Export' and save file to your project directory

<img src="https://github.com/strmwtr/downtown_assessments/blob/master/img/getting_pin_list.png?raw=true">

# Step 2: Prepare Data

## Looking at the .xls retrieved from the GIS Viewer

In [1]:
#Import pandas module
import pandas as pd 

#Path to the .xls retrieve from the GIS Viewer
f = r'./data/pin_exp.xls'

#Create a dataframe that reads the .xls file
df = pd.read_excel(f)

We can quickly see what is provided by this .xls sheet and the general format of the data it contains. We can see that there are 23 columns by the 5 rows x 23 columns line printed at the end of .head(). 

Looking at the column headers

In [2]:
print("df.head()\n", df.head())
print('-'*80)
print("\ndf.columns:\n", df.columns)
print('-'*80)
print("\ndf['PIN'].head():\n", df['PIN'].head())
print('-'*80)
print("\ndf['PIN'].shape:\n",df['PIN'].shape)
print('-'*80)
print("\ndf['PIN'].unique():\n", df['PIN'].unique())
print('-'*80)
print("\ndf['PIN'].unique().shape:", df['PIN'].unique().shape)

df.head()
                   FullAddress  OBJECTID        PIN   GPIN ParcelNumber  \
0                0 3RD ST SE   24846953  280036300   7309    280036300   
1              0 E MARKET ST   24848655  330245100  14744    330245100   
2  0 W MARKET ST & 2ND ST NW   24841086  330262000   6656    330262000   
3              100 5TH ST SE   24845005  530065300   7426    530065300   
4              100 E MAIN ST   24839773  280020000   7021    280020000   

                                     OwnerName  CurrentAssessedValue  \
0                            LITTLE MOOSE, LLC                204100   
1           FIRST AND MAIN CHARLOTTESVILLE LLC               1122200   
2  SPENCER, HAWES, ETAL, TR PROTICO PROP LD TR                418400   
3           MAIN, RALPH TR OF BLACK DUCK LD TR                664000   
4               ONE HUNDRED EAST MAIN LTD PART               1904200   

  CurrentTaxYear CurrentAssessedValueWithLabel  \
0    2019 Value:          2019 Value:  204,100   
1    2019 V

Now we have a list of unique PIN's that we can use to build our desired Assessments over time dataframe from. Let's set this to it's own dataframe so we can easily call it in the future

In [3]:
#Create dataframe unique_pins_df from unique PIN's in df  
unique_pins_df = pd.DataFrame(df['PIN'].unique())
#Set column name to PIN
unique_pins_df.columns = ['PIN']
unique_pins_df.head()

,PIN
0,280036300
1,330245100
2,330262000
3,530065300
4,280020000


In [30]:
#Create dataframe unique_pins_df from unique PIN's in df  
unique_address_df = pd.DataFrame(df['FullAddress'].unique())
#Set column name to PIN
unique_address_df.columns = ['FullAddress']
unique_address_df.head()

,FullAddress
0,0 3RD ST SE
1,0 E MARKET ST
2,0 W MARKET ST & 2ND ST NW
3,100 5TH ST SE
4,100 E MAIN ST


## Preparing annual assessment data

Charlottesville's Open Data Portal : http://opendata.charlottesville.org/

Real Estate (All Assessments) Dataset : http://opendata.charlottesville.org/datasets/real-estate-all-assessments

* On the Real Estate dataset page, in the upper right corner of the window under the map click the 'API Explorer' tab
* Copy the Query URL link and augment the link to match your query
* Use the augmented link to pull data directly from the Open Data portal using the code below

In [21]:
# importing the requests library 
import requests

formatted_pins = [f'%27{x}%27' for x in df['PIN'].unique()]
formatted_pins_1 = formatted_pins[:75]
formatted_pins_2 = formatted_pins[75:]
formatted_pins_1 = ','.join(formatted_pins_1)
formatted_pins_2 = ','.join(formatted_pins_2)

url1 = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins_1})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear,StreetName,StreetNumber,Unit&outSR=4326&f=json"
url2 = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins_2})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear,StreetName,StreetNumber,Unit&outSR=4326&f=json"

r1 = requests.get(url1)
r2 = requests.get(url2)

d1 = r1.json()
d2 = r2.json()
print(r1,r2)

<Response [200]> <Response [200]>


In [32]:
# importing the requests library 
import requests

formatted_pins = [f'%27{x}%27' for x in unique_address_df]
formatted_pins_1 = formatted_pins[:75]
formatted_pins_2 = formatted_pins[75:]
formatted_pins_1 = ','.join(formatted_pins_1)
formatted_pins_2 = ','.join(formatted_pins_2)

url1 = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins_1})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear,StreetName,StreetNumber,Unit&outSR=4326&f=json"
url2 = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins_2})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear,StreetName,StreetNumber,Unit&outSR=4326&f=json"
print(url1)
r1 = requests.get(url1)
r2 = requests.get(url2)

d1 = r1.json()
d2 = r2.json()
print(r1,r2)

https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20(%27FullAddress%27)%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear,StreetName,StreetNumber,Unit&outSR=4326&f=json
<Response [200]> <Response [200]>


After testing the requests.get(url), I can request up to 120 parcels at a time before recieving a 404 Error. I have 126 parcels of interest. I will break my request up into 2 parts, [:75] and [75:].

On a future date I will write a function that checks for responses under 400 based on length of results. This will eliminate the need for writing out formatted_pins_1 .. 2 .. 3. 

Check data type

In [22]:
#Check data type
print(type(d1))

print()
#Check keys
print(d1.keys())

print()
#check features key
print(d1['features'][:5])

print()
#Check type of d1['features']
type(d1['features'])



<class 'dict'>

dict_keys(['displayFieldName', 'fieldAliases', 'fields', 'features'])

[{'attributes': {'ParcelNumber': '280010000', 'LandValue': 2191100, 'ImprovementValue': 3812400, 'TotalValue': 6003500, 'TaxYear': '2019', 'StreetName': 'W MAIN ST', 'StreetNumber': '200', 'Unit': ''}}, {'attributes': {'ParcelNumber': '280010000', 'LandValue': 2140600, 'ImprovementValue': 4038400, 'TotalValue': 6179000, 'TaxYear': '2018', 'StreetName': 'W MAIN ST', 'StreetNumber': '200', 'Unit': ''}}, {'attributes': {'ParcelNumber': '280010000', 'LandValue': 2140600, 'ImprovementValue': 4051208, 'TotalValue': 6191808, 'TaxYear': '2017', 'StreetName': 'W MAIN ST', 'StreetNumber': '200', 'Unit': ''}}, {'attributes': {'ParcelNumber': '280010000', 'LandValue': 1719500, 'ImprovementValue': 4271200, 'TotalValue': 5990700, 'TaxYear': '2016', 'StreetName': 'W MAIN ST', 'StreetNumber': '200', 'Unit': ''}}, {'attributes': {'ParcelNumber': '280010000', 'LandValue': 1563200, 'ImprovementValue': 1120000, 'TotalVa

list

## Create date series based on features and combine data frames into a single df

In [23]:
df1 = pd.DataFrame(d1['features'])
df2 = pd.DataFrame(d2['features'])
print('.shape of df1, df2: ', df1.shape, df2.shape)
df1 = df1.append(df2, ignore_index = True)
print('.shape of df1 after appending df2: ', df1.shape)

print('\ndf1.head(): \n',df1.head())

print('\ndf1.keys(): ',df1.keys())

print('\ntype(df1["attributes"]): ',type(df1['attributes']))

.shape of df1, df2:  (1683, 1) (1173, 1)
.shape of df1 after appending df2:  (2856, 1)

df1.head(): 
                                           attributes
0  {'ParcelNumber': '280010000', 'LandValue': 219...
1  {'ParcelNumber': '280010000', 'LandValue': 214...
2  {'ParcelNumber': '280010000', 'LandValue': 214...
3  {'ParcelNumber': '280010000', 'LandValue': 171...
4  {'ParcelNumber': '280010000', 'LandValue': 156...

df1.keys():  Index(['attributes'], dtype='object')

type(df1["attributes"]):  <class 'pandas.core.series.Series'>


## Create a single data frame based on combined series data

In [24]:
assessments = pd.DataFrame([x for x in df1['attributes']])
assessments.head()

,ImprovementValue,LandValue,ParcelNumber,StreetName,StreetNumber,TaxYear,TotalValue,Unit
0,3812400,2191100,280010000,W MAIN ST,200,2019,6003500,
1,4038400,2140600,280010000,W MAIN ST,200,2018,6179000,
2,4051208,2140600,280010000,W MAIN ST,200,2017,6191808,
3,4271200,1719500,280010000,W MAIN ST,200,2016,5990700,
4,1120000,1563200,280010000,W MAIN ST,200,2015,2683200,


In [25]:
print('assessments.keys():\n',assessments.keys())

print('\nassessments.shape: ', assessments.shape)

print('\nassessments.isnull().any():\n', assessments.isnull().any())

assessments.keys():
 Index(['ImprovementValue', 'LandValue', 'ParcelNumber', 'StreetName',
       'StreetNumber', 'TaxYear', 'TotalValue', 'Unit'],
      dtype='object')

assessments.shape:  (2856, 8)

assessments.isnull().any():
 ImprovementValue    False
LandValue           False
ParcelNumber        False
StreetName          False
StreetNumber        False
TaxYear             False
TotalValue          False
Unit                False
dtype: bool


print("\nassessments['ParcelNumber'].describe()\n", assessments['ParcelNumber'].describe())

print("\nassessments['TaxYear'].describe()\n", assessments['TaxYear'].describe())

print("\nassessments['TaxYear'].min(), assessments['TaxYear'].max()\n", assessments['TaxYear'].min(), assessments['TaxYear'].max())

print("\nassessments['ImprovementValue'].describe()\n", assessments['ImprovementValue'].describe())

print("\nassessments['LandValue'].describe()\n", assessments['LandValue'].describe())

print("\nassessments['TotalValue'].describe()\n", assessments['TotalValue'].describe())

taxyearmin = assessments['TaxYear'] == assessments['TaxYear'].min()
assessments[taxyearmin].describe()

taxyearmax = assessments['TaxYear'] == assessments['TaxYear'].max()
assessments[taxyearmax].describe()

## assessments[taxyearmax].describe()-assessments[taxyearmin].describe()

## Access json file for parcel areas

import folium

formatted_gpins = [str(x) for x in df['GPIN'].unique()]
formatted_gpins = formatted_gpins
formatted_gpins = ','.join(formatted_gpins)

parcel_area_url = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_1/MapServer/43/query?where=GPIN%20in%20({formatted_gpins})&outFields=*&outSR=4326&f=json"

print(parcel_area_url)

m = folium.Map(location=[38.0309,-78.4804],tiles='Stamen Terrain',zoom_start=17)
style_function = {'fillOpacity': 0, 'fillColor': '#black'}
folium.GeoJson(parcel_area_url,name='Parcels', style_function=style_function).add_to(m)

folium.LayerControl().add_to(m)
m



m = folium.Map(location=[38.0309,-78.4804],tiles='Stamen Terrain',zoom_start=17)

folium.Choropleth(
    geo_data=parcel_area_url,
    name='choropleth',
    data=assessments,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [29]:
df_key = df[['PIN','GPIN','FullAddress']]
df_key
assessments

,ImprovementValue,LandValue,ParcelNumber,StreetName,StreetNumber,TaxYear,TotalValue,Unit
0,3812400,2191100,280010000,W MAIN ST,200,2019,6003500,
1,4038400,2140600,280010000,W MAIN ST,200,2018,6179000,
2,4051208,2140600,280010000,W MAIN ST,200,2017,6191808,
3,4271200,1719500,280010000,W MAIN ST,200,2016,5990700,
4,1120000,1563200,280010000,W MAIN ST,200,2015,2683200,
5,1120000,1563200,280010000,W MAIN ST,200,2014,2683200,
6,1120000,1488800,280010000,W MAIN ST,200,2013,2608800,
7,1805000,1198700,280010000,W MAIN ST,200,2012,3003700,
8,1805000,1198700,280010000,W MAIN ST,200,2011,3003700,
9,1805000,1198700,280010000,W MAIN ST,200,2010,3003700,


In [11]:
assessments.head()

,ImprovementValue,LandValue,ParcelNumber,TaxYear,TotalValue
0,3812400,2191100,280010000,2019,6003500
1,4038400,2140600,280010000,2018,6179000
2,4051208,2140600,280010000,2017,6191808
3,4271200,1719500,280010000,2016,5990700
4,1120000,1563200,280010000,2015,2683200


In [12]:
pin_count = df_key.groupby('PIN').count() 
pin_count

,GPIN
PIN,
280001000,1
280010000,1
280012000,1
280013000,1
280016000,10
280016100,1
280018000,1
280019000,1
280020000,1


In [13]:
multi_pin = pin_count[pin_count.GPIN != 1]
multi_pin

,GPIN
PIN,
280016000,10
330257100,4
530054101,27
530055000,2
530064000,2
530072000,23
530091000,17
280037A00,4


In [20]:
a = assessments[assessments.ParcelNumber == '530054101']
b = a[a.TaxYear == '2019']
b

,ImprovementValue,LandValue,ParcelNumber,TaxYear,TotalValue
2373,282900,91300,530054101,2019,374200


In [36]:
df[df.MULTIPIN == 1.0]

,FullAddress,OBJECTID,PIN,GPIN,ParcelNumber,OwnerName,CurrentAssessedValue,CurrentTaxYear,CurrentAssessedValueWithLabel,PicturePath,...,MULTIPIN,OwnerAddress,OwnerCityState,OwnerZipCode,SHAPE.STArea(),SHAPE.STLength(),cvGIS.CITY.parcel_area.CreatedBy,cvGIS.CITY.parcel_area.CreatedDate,cvGIS.CITY.parcel_area.ModifiedBy,cvGIS.CITY.parcel_area.ModifiedDate
118,Various,24841105,330257100,6778,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,http://realestate.charlottesville.org/IMAGES\P...,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,3253.750000,318.833585,NaN,NaN,CHAMBERST,"2/16/18, 5:25:13 PM"
119,Various,24841106,330257100,6778,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,http://realestate.charlottesville.org/IMAGES\P...,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,3253.750000,318.833585,NaN,NaN,CHAMBERST,"2/16/18, 5:25:13 PM"
120,Various,24841107,330257100,6778,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,http://realestate.charlottesville.org/IMAGES\P...,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,3253.750000,318.833585,NaN,NaN,CHAMBERST,"2/16/18, 5:25:13 PM"
121,Various,24841108,330257100,6778,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,http://realestate.charlottesville.org/IMAGES\P...,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,3253.750000,318.833585,NaN,NaN,CHAMBERST,"2/16/18, 5:25:13 PM"
122,Various,24851192,530055000,6998,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,http://realestate.charlottesville.org/IMAGES\P...,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,3902.250000,252.143495,NaN,NaN,CHAMBERST,"2/16/18, 5:09:44 PM"
123,Various,24851191,530055000,6998,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,NaN,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,3902.250000,252.143495,NaN,NaN,CHAMBERST,"2/16/18, 5:09:44 PM"
124,Various,24849215,530054101,7023,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,http://realestate.charlottesville.org/IMAGES\P...,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,15578.250000,500.485048,NaN,NaN,CHAMBERST,"2/16/18, 5:09:44 PM"
125,Various,24849216,530054101,7023,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,http://realestate.charlottesville.org/IMAGES\P...,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,15578.250000,500.485048,NaN,NaN,CHAMBERST,"2/16/18, 5:09:44 PM"
126,Various,24849217,530054101,7023,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,http://realestate.charlottesville.org/IMAGES\P...,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,15578.250000,500.485048,NaN,NaN,CHAMBERST,"2/16/18, 5:09:44 PM"
127,Various,24849218,530054101,7023,Multiple Parcels,Multiple Owners,0,2019 Value:,Multiple Values,http://realestate.charlottesville.org/IMAGES\P...,...,1.0,Multiple Owners,Multiple Owners,Multiple Owners,15578.250000,500.485048,NaN,NaN,CHAMBERST,"2/16/18, 5:09:44 PM"


# Exclude all multi pin properties from assessments df

# Pull all multi pin properties from xls
# Query assessment values based on street name
# Combine multi pin into 1 record per tax year
# Remove all multi pin data from assessments df
# Add combined multi pin data back into assessments

# Join df with assessments

# Join Parcel Area with assessments

# Plot each parcels total, land, and improvement value across all years on 3 line graphs, one for each assessment type

# Map the same data as above via folium

# Create time lapse of maps